In [16]:
### 10.3.3
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# set executable path and set up the URL:

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\bdydh\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [3]:
# In the next cell of your Jupyter notebook, we'll assign the url and instruct the browser to visit it.
# One is that we're searching for elements with a specific combination of tag (div) and attribute (list_text). 
# Secondly, we're also telling our browser to wait one second before searching for components
# Visit the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [4]:
# set up the HTML parser:
## The . is used for selecting classes, such as list_text, so the code 'div.list_text' pinpoints the <div /> tag 
# with the class of list_text
# In this line of code, we chained .find onto our previously assigned variable, slide_elem. When we do this, 
# we're saying, "This variable holds a ton of information, so look inside of that information to find this 
# specific data." The data we're looking for is the content title, which we've specified by saying, 
# "The specific data is in a <div /> with a class of 'content_title'."


html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [5]:
# We'll want to assign the title and summary text to variables we'll reference later. In the next empty cell, 
# let's begin our scraping. Type the following:

slide_elem.find('div', class_='content_title')

<div class="content_title">NASA to Hold Mars 2020 Perseverance Rover Launch Briefing</div>

In [6]:
# we need to get just the text, and the extra HTML stuff isn't necessary. In the next cell, type the following:

# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

'NASA to Hold Mars 2020 Perseverance Rover Launch Briefing'

In [7]:
#There are two methods used to find tags and attributes with BeautifulSoup:
#.find() is used when we want only the first class and attribute we've specified.
#.find_all() is used when we want to retrieve all of the tags and attributes.

# Use the parent element to find the paragraph text

news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

"Learn more about the agency's next Red Planet mission during a live event on June 17."

### 10.3.4
# In your Jupyter notebook, use markdown to separate the article scraping 
# from the image scraping.
# change format of code to markdown on the top menu (where you reset kernel # etc)

### Featured Images


In [8]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [10]:
# Find and click the full image button
# Notice the indexing chained at the end of the first line of code? 
# With this, we've stipulated that we want our browser to click the second button.

full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [11]:
## With the new page loaded onto our automated browser, it needs to be parsed 
## so we can continue and scrape the full-size image URL.

# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [13]:
## We'll use the image tag and class (<img />and fancybox-img) to build the URL to the full-size image.
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars2.jpg'

In [14]:
# Let's add the base URL to our code.
# We're using an f-string for this print statement because it's a cleaner way to create print statements; 
# they're also evaluated at run-time. 
# Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [17]:
### 10.3.5
# Instead of scraping each row, or the data in each <td />, we're going to scrape the entire table 
# with Pandas' .read_html() function.
# With this line, we're creating a new DataFrame from the HTML table.
# we assign columns to the new DataFrame for additional clarity.
# By using the .set_index() function, we're turning the Description column into the 
# DataFrame's index. inplace=True means that the updated index will remain in place, 
# without having to reassign the DataFrame to a new variable.

df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [18]:
# Pandas also has a way to easily convert our DataFrame back into HTML-ready code using the .to_html() function
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [19]:
# Turn off the browser so it does not drain your laptop battery etc
browser.quit()